# Text Generation using GPT (Using Huggingface)

## Project Setup

In [1]:
!pip install -q transformers

In [6]:
conda install pytorch torchvision torchaudio -c pytorch-nightly

Solving environment: done

## Package Plan ##

  environment location: /opt/homebrew/Caskroom/miniconda/base

  added / updated specs:
    - pytorch
    - torchaudio
    - torchvision


The following packages will be UPDATED:

  ca-certificates    conda-forge::ca-certificates-2023.5.7~ --> pkgs/main::ca-certificates-2023.05.30-hca03da5_0 

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi            conda-forge/noarch::certifi-2023.5.7-~ --> pkgs/main/osx-arm64::certifi-2023.5.7-py310hca03da5_0 




Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
# import math# this ensures that the current MacOS version is at least 12.3+
print(torch.backends.mps.is_available())# this ensures that the current current PyTorch installation was built with MPS activated.
print(torch.backends.mps.is_built())

True
True


In [1]:
import torch
import shutil
from torch.utils.data import Dataset, random_split
from transformers import Trainer, TrainingArguments, GPTNeoForCausalLM, GPT2Tokenizer





## Data Preparation

In [2]:
# Load data into colab
!wget https://huggingface.co/datasets/vindhyamganti/Ted/resolve/main/transcripts.txt

--2023-06-20 13:02:54--  https://huggingface.co/datasets/vindhyamganti/Ted/resolve/main/transcripts.txt
Resolving huggingface.co (huggingface.co)... 2600:9000:254b:fe00:8:2a4d:8540:93a1, 2600:9000:254b:9200:8:2a4d:8540:93a1, 2600:9000:254b:1200:8:2a4d:8540:93a1, ...
Connecting to huggingface.co (huggingface.co)|2600:9000:254b:fe00:8:2a4d:8540:93a1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/3e/22/3e2292ba59d5756686370a24a97902a7f9ec0acb947e8abec79a3ab6fc563d90/9fc6c11b5576f38d762d797d543f4ba0aea3ce5d0cda952838a3bf9852e33c3f?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27transcripts.txt%3B+filename%3D%22transcripts.txt%22%3B&response-content-type=text%2Fplain&Expires=1687543375&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzNlLzIyLzNlMjI5MmJhNTlkNTc1NjY4NjM3MGEyNGE5NzkwMmE3ZjllYzBhY2I5NDdlOGFiZWM3OWEzYWI2ZmM1NjNkOTAvOWZjNmMxMWI1NTc2ZjM4ZDc2MmQ3OTdkNTQz

In [16]:
# from google.colab import drive
# # Connects colab to google drive
# drive.mount('/content/drive')
import os

# Mount Google Drive (Colab-specific code)
# from google.colab import drive
# drive.mount('/content/drive')

# Access files in Google Drive (Modified code)
file_path = 'combined_datasets.txt'


In [ ]:
# shutil.copy("/content/transcripts.txt.2","drive/MyDrive/AICamp/nlp")
# 

'drive/MyDrive/AICamp/nlp/transcripts.txt.2'

In [17]:
# Read the text file and returns list of lines in text
def read_file(file_path):
    with open(file_path) as f:
        lines = [line for line in f]
        # lines.remove("")
    return lines


In [18]:
# file_path = "drive/MyDrive/AICamp/nlp/transcripts.txt"

texts = read_file(file_path)
print(texts[1])
# transcripts = []
# transcript = []
# for text in texts:
#   if len(text)>1:
#     transcript.append(text)
#   else:
#     transcripts.append(''.join(transcript))
#     transcript = []

# Remove unnecessary texts


<End of Text>



In [19]:
# Prepare sonnets
datas = texts[2:-1]
print(len(datas))
for data in datas:
    if len(data)<1:
        datas.remove(data)
print(len(datas))

12913
12913


In [24]:
# Custome dataset class to load dataset
class TranscriptsDataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in txt_list:
            # Encode the descriptions using the GPT-Neo tokenizer
            encodings_dict = tokenizer('<|startoftext|>'
                                        + txt +
                                        '<|endoftext|>',
                                        truncation=True,
                                        max_length=max_length,
                                            padding="max_length")
            input_ids = torch.tensor(encodings_dict['input_ids'])
            self.input_ids.append(input_ids)
            mask = torch.tensor(encodings_dict['attention_mask'])
            self.attn_masks.append(mask)

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

## Initialize tokenizer, model

In [31]:
torch.cuda.is_available()

False

In [25]:
# Set the random seed to a fixed value to get reproducible results
torch.manual_seed(42)

# Download the pre-trained GPT-Neo model's tokenizer
# Add the custom tokens denoting the beginning and the end
# of the sequence and a special token for padding
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-125M",
                            bos_token='<|startoftext|>',
                            eos_token='<|endoftext|>',
                            pad_token='<|pad|>')

# Download the pre-trained GPT-Neo model and transfer it to the GPU
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M")

# Resize the token embeddings because we've just added 3 new tokens
model.resize_token_embeddings(len(tokenizer))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Embedding(50259, 768)

## Train/Test Split data

In [28]:
# max_length = max([len(tokenizer.encode(sonnet)) for transcript in datas])

# Load dataset
dataset = TranscriptsDataset(texts, tokenizer, 2000)

# Split data into train/val
train_size = int(0.9 * len(dataset))

train_data, val_data = random_split(dataset, [train_size, len(dataset) - train_size])

# max_length

In [29]:
tokenizer.batch_decode(val_data[0])

['<|startoftext|> I\'m going to tell you a little bit about my TEDxHouston Talk. I woke up the morning after I gave that talk with the worst vulnerability hangover of my life. And I actually didn\'t leave my house for about three days. The first time I left was to meet a friend for lunch. And when I walked in, she was already at the table. I sat down, and she said, "God, you look like hell." I said, "Thanks. I feel really - I\'m not functioning." And she said, "What\'s going on?" And I said, "I just told 500 people that I became a researcher to avoid vulnerability. And that when being vulnerable emerged from my data, as absolutely essential to whole-hearted living, I told these 500 people that I had a breakdown. I had a slide that said \'Breakdown.\' At what point did I think that was a good idea?" And she said, "I saw your talk live-streamed. It was not really you. It was a little different than what you usually do. But it was great." And I said, "This can\'t happen. YouTube, they\'re

## Train Model

In [ ]:
!pip install accelerate -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
# Here I will pass the output directory where
# the model predictions and checkpoints will be stored,
# batch sizes for the training and validation steps,
# and warmup_steps to gradually increase the learning rate
learning_rates = [5e-5, 3e-5, 1e-5]


for learning_rate in learning_rates:

    training_args = TrainingArguments(output_dir=f'./results_{learning_rate}',
                                      num_train_epochs=5,
                                      logging_steps=1000,
                                      save_steps=1000,
                                      evaluation_strategy='steps',
                                      eval_steps=1000,
                                      per_device_train_batch_size=2,
                                      per_device_eval_batch_size=2,
                                      warmup_steps=100,
                                      learning_rate=learning_rate,
                                      weight_decay=0.01,
                                      logging_dir=f'./logs_{learning_rate}')

    trainer = Trainer(model=model, args=training_args,
                      train_dataset=train_data,
                      eval_dataset=val_data,
                      # This custom collate function is necessary
                      # to built batches of data
                      data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                  'attention_mask': torch.stack([f[1] for f in data]),
                  'labels': torch.stack([f[0] for f in data])})

    # Start training process!
    print(f"Training result for learning rate: {learning_rate}")
    trainer.train()
    print("\n\n")

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

BAsed on the results above, it looks like model trained with learning rate = 5e-5 is more promising than others.

In [ ]:
training_args = TrainingArguments(output_dir=f'./results',
                                      num_train_epochs=5,
                                      logging_steps=1000,
                                      save_steps=5000,
                                      evaluation_strategy='steps',
                                      eval_steps=1000,
                                      per_device_train_batch_size=2,
                                      per_device_eval_batch_size=2,
                                      warmup_steps=100,
                                      learning_rate=5e-5,
                                      weight_decay=0.01,
                                      logging_dir=f'./logs')

trainer = Trainer(model=model, args=training_args,
                  train_dataset=train_data,
                  eval_dataset=val_data,
                  # This custom collate function is necessary
                  # to built batches of data
                  data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
              'attention_mask': torch.stack([f[1] for f in data]),
              'labels': torch.stack([f[0] for f in data])})

# Start training process!
trainer.train()


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 24>:24                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1645 in train                    │
│                                                                                                  │
│   1642 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1643 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1644 │   │   )                                                                                 │
│ ❱ 1645 │   │   return inner_training_loop(                                                       │
│   1646 │   │   │   args=args,                                                                    │
│   1647 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1648 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1938 in _inner_training_loop     │
│                                                                                                  │
│   1935 │   │   │   │   │   self.control = self.callback_handler.on_step_begin(args, self.state,  │
│   1936 │   │   │   │                                                                             │
│   1937 │   │   │   │   with self.accelerator.accumulate(model):                                  │
│ ❱ 1938 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1939 │   │   │   │                                                                             │
│   1940 │   │   │   │   if (                                                                      │
│   1941 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2770 in training_step            │
│                                                                                                  │
│   2767 │   │   │   with amp.scale_loss(loss, self.optimizer) as scaled_loss:                     │
│   2768 │   │   │   │   scaled_loss.backward()                                                    │
│   2769 │   │   else:                                                                             │
│ ❱ 2770 │   │   │   self.accelerator.backward(loss)                                               │
│   2771 │   │                                                                                     │
│   2772 │   │   return loss.detach() / self.args.gradient_accumulation_steps                      │
│   2773                                                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:1821 in backward               │
│                                                                                                  │
│   1818 │   │   elif self.scaler is not None:                                                     │
│   1819 │   │   │   self.scaler.scale(loss).backward(**kwargs)                                    │
│   1820 │   │   else:                                                                             │
│ ❱ 1821 │   │   │   loss.backward(**kwargs)                                                       │
│   1822 │                                                   

In [ ]:
# Save model in the specified file path
trainer.save_model("drive/MyDrive/AICamp/nlp/")

In [ ]:
tokenizer.save_pretrained("drive/MyDrive/AICamp/nlp/")

('drive/MyDrive/AICamp/nlp/tokenizer_config.json',
 'drive/MyDrive/AICamp/nlp/special_tokens_map.json',
 'drive/MyDrive/AICamp/nlp/vocab.json',
 'drive/MyDrive/AICamp/nlp/merges.txt',
 'drive/MyDrive/AICamp/nlp/added_tokens.json')

## Checking Model Output

In [ ]:
generated = tokenizer("<|startoftext|>", return_tensors="pt").input_ids.cuda()
sample_outputs = model.generate(generated, do_sample=True, top_k=50,
                                # bos_token='<|startoftext|>',
                                # eos_token='<|endoftext|>', pad_token='<|pad|>',
                                max_length=300, top_p=0.95, temperature=1.9, num_return_sequences=20)
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0: 
1: 
2: Where should that evil cease, or those things would return to their owners'?
To prove their good grace (That thou must find worthy still remains unhearest, thou must seek this end, and that this which thine (Nor if this be so thy worst must be my joy)?
Let me return not once nor hold still an unseeing eye
A pity where's this coming help bestow thee?
Then thine eye be thy back-ticks where it most hinds theego
Where reason guides both minds whence thou wost and me to seek!?


3: In the earth it holds but gentleclips and jewels,
Lest things that belong alone, should themselves boast,
The precious parts so grounded to nature'er line,   
Without having checked this abundance themselves bear,   
SIXth many miles in motion from the desert place.   
Nor are stars in all kinds of stars less grounded,
They dwell on me by giving account to my skill:
And though some are not more great or more than others,
More even or even all their compass height than others.
All this hath in the world

In [ ]:
! transformers-cli env

2023-03-24 22:53:44.974369: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-24 22:53:44.974575: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-24 22:53:44.974603: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
2023-03-24 22:53:55.043416: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:

## Upload model to huggingface

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from huggingface_hub import HfApi

api = HfApi()

In [ ]:
# Create your repo first to upload the model
api.create_repo(repo_id="gpt2-sonnet-generators")

RepoUrl('https://huggingface.co/niki-stha/gpt2-sonnet-generators', endpoint='https://huggingface.co', repo_type='model', repo_id='niki-stha/gpt2-sonnet-generators')

In [ ]:
# Upload your model to huggingface. You can clone the repo anytime to use the model.
import os

model_pth = "drive/MyDrive/AICamp/nlp/models"

files = os.listdir(model_pth)
for fi in files:
    print(os.path.join(model_pth, fi))

    api.upload_file(
        path_or_fileobj=os.path.join(model_pth, fi),
        path_in_repo=fi,
        repo_id="niki-stha/gpt2-sonnet-generators",
        repo_type="model",
    )

drive/MyDrive/AICamp/nlp/models/config.json
drive/MyDrive/AICamp/nlp/models/generation_config.json
drive/MyDrive/AICamp/nlp/models/pytorch_model.bin


Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

drive/MyDrive/AICamp/nlp/models/training_args.bin


Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/3.52k [00:00<?, ?B/s]

drive/MyDrive/AICamp/nlp/models/tokenizer_config.json
drive/MyDrive/AICamp/nlp/models/special_tokens_map.json
drive/MyDrive/AICamp/nlp/models/added_tokens.json
drive/MyDrive/AICamp/nlp/models/vocab.json
drive/MyDrive/AICamp/nlp/models/merges.txt
